# Machine Learning Example Notebook

This notebook demonstrates basic ML operations with numpy, torch, and transformers.

In [ ]:
# Import necessary libraries
import numpy as np
import torch
import matplotlib.pyplot as plt
from transformers import BertConfig, BertModel
import sys
import os

# Add the src directory to the Python path
sys.path.insert(0, os.path.abspath('..'))

# Import from our project
from {{cookiecutter.project_slug}}.ml_example import numpy_calculations, torch_calculations, plot_data

# Set up matplotlib for inline display
%matplotlib inline
plt.style.use('ggplot')

## 1. Numpy Calculations

Let's perform some basic calculations using numpy.

In [ ]:
# Run numpy calculations
numpy_data = numpy_calculations()

# Additional numpy operations
# Create a random matrix with specific shape
X = np.random.rand(100, 10)
y = np.random.randint(0, 2, size=100)  # Binary classification for example

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")

# Show a sample
print(f"Sample features:{X[:5, :5]}")
print(f"Sample labels: {y[:10]}")

# Class distribution
unique, counts = np.unique(y, return_counts=True)
print(f"Class distribution: {dict(zip(unique, counts))}")

## 2. PyTorch Calculations

Now let's perform similar calculations using PyTorch.

In [ ]:
# Run torch calculations
tensor, torch_output = torch_calculations()

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

print(f"X tensor shape: {X_tensor.shape}")
print(f"y tensor shape: {y_tensor.shape}")

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)
print(f"Using device: {device}")

# Create a simple neural network
model = torch.nn.Sequential(
    torch.nn.Linear(X.shape[1], 20),
    torch.nn.ReLU(),
    torch.nn.Linear(20, 2)
).to(device)

# Forward pass
outputs = model(X_tensor)
print(f"Model output shape: {outputs.shape}")
print(f"First few outputs:{outputs[:5]}")

## 3. Visualization with Matplotlib

Let's create some visualizations of our data.

In [ ]:
# Use the plot_data function from ml_example.py
plot_data(numpy_data, outputs)

# Create additional custom plots
plt.figure(figsize=(12, 8))

# Plot 1: Heatmap of numpy matrix
plt.subplot(2, 2, 1)
plt.imshow(X[:10, :10], cmap='viridis')
plt.colorbar()
plt.title('Feature Matrix Heatmap')

# Plot 2: Line plot of numpy data
plt.subplot(2, 2, 2)
for i in range(5):  # First 5 features
    plt.plot(X[:20, i], label=f'Feature {i+1}')  # First 20 samples
plt.title('Feature Values')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()

# Plot 3: Heatmap of model outputs
plt.subplot(2, 2, 3)
plt.imshow(outputs[:10].detach().cpu().numpy(), cmap='plasma')
plt.colorbar()
plt.title('Model Outputs Heatmap')

# Plot 4: Bar chart of class distribution
plt.subplot(2, 2, 4)
plt.bar(unique, counts)
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Count')

plt.tight_layout()

## 4. Loading a Transformers Model from Custom Config

Now let's create a small untrained BERT model from a custom configuration.

In [ ]:
# Create a small transformer model with custom configuration
config = BertConfig(
    vocab_size=1000,          # Reduced vocabulary size
    hidden_size=64,           # Smaller hidden size
    num_hidden_layers=2,      # Fewer layers
    num_attention_heads=2,    # Fewer attention heads
    intermediate_size=256,    # Smaller intermediate size
    max_position_embeddings=512
)

model = BertModel(config)

# Print the configuration
print("Custom BERT Configuration:")
print(config)

In [ ]:
# Print model summary
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# List some of the model's modules
for name, module in list(model.named_modules())[:10]:  # Show only first 10 modules
    print(f"{name}: {module.__class__.__name__}")

In [ ]:
# Create a sample input
batch_size = 2
seq_length = 10
input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_length))
attention_mask = torch.ones(batch_size, seq_length)

print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention mask shape: {attention_mask.shape}")

In [ ]:
# Run the model
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Examine the outputs
last_hidden_state = outputs.last_hidden_state
pooler_output = outputs.pooler_output

print(f"Last hidden state shape: {last_hidden_state.shape}")
print(f"Pooler output shape: {pooler_output.shape}")

In [ ]:
# Visualize the embeddings
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot the last hidden state for the first sequence
im = axes[0].imshow(last_hidden_state[0].detach().cpu().numpy(), cmap='viridis')
plt.colorbar(im, ax=axes[0])
axes[0].set_title('Last Hidden State (First Sequence)')
axes[0].set_xlabel('Hidden Dimension')
axes[0].set_ylabel('Sequence Position')

# Plot the pooler output
axes[1].bar(range(pooler_output[0].shape[0]), pooler_output[0].detach().cpu().numpy())
axes[1].set_title('Pooler Output (First Sequence)')
axes[1].set_xlabel('Hidden Dimension')
axes[1].set_ylabel('Value')

plt.tight_layout()

## Conclusion

In this notebook, we've demonstrated:

1. Basic calculations with NumPy
2. Similar operations with PyTorch
3. Data visualization with Matplotlib
4. Creating and using a small untrained transformer model from a custom configuration

These examples show how to use the core libraries included in this ML project template.
